##### 1. Import Libraries

In [6]:
import pandas as pd
import numpy as np


##### 2. Load Walmart & Amazon Datasets

In [7]:
walmart= pd.read_csv("walmart_products.csv")
amazon_raw = pd.read_csv(
    "amazon_products.csv",
    engine="python",
    sep=",",
    on_bad_lines="skip"
)


print('amazon-shape:', amazon_raw.shape)

amazon-shape: (1066, 11220)


In [8]:
# Remove columns with no name
amazon = amazon_raw.loc[:, ~amazon_raw.columns.str.contains("^Unnamed")]

# Remove fully empty columns
amazon = amazon.dropna(axis=1, how="all")

print("After cleaning:", amazon.shape)


After cleaning: (1066, 85)


##### 3. Checks shapes

In [9]:
print("Walmart:", walmart.shape)
print("Amazon:", amazon.shape)

Walmart: (1000, 44)
Amazon: (1066, 85)


##### 4. Inspecting amazon file format

In [10]:
# View first few lines raw
with open("amazon_products.csv", "r", encoding="utf-8") as f:
    for i in range(5):
        print(f.readline())


title,seller_name,brand,description,initial_price,currency,availability,reviews_count,categories,parent_asin,asin,buybox_seller,number_of_sellers,root_bs_rank,ISBN10,answered_questions,domain,images_count,url,video_count,image_url,item_weight,rating,product_dimensions,seller_id,image,date_first_available,discount,model_number,manufacturer,department,plus_content,upc,video,top_review,final_price_high,final_price,variations,delivery,features,format,buybox_prices,input_asin,ingredients,origin_url,bought_past_month,is_available,root_bs_category,bs_category,bs_rank,badge,subcategory_rank,amazon_choice,images,product_details,prices_breakdown,country_of_origin,from_the_brand,product_description,seller_url,customer_says,sustainability_features,climate_pledge_friendly,videos,other_sellers_prices,downloadable_videos,editorial_reviews,about_the_author,zipcode,coupon,sponsored,store_url,ships_from,city,customers_say,max_quantity_available,variations_values,language,return_policy,inactive_buy_box,b

##### Standardize Amazon Columns to Match Walmart

In [11]:
amazon = amazon.rename(columns={
    "price": "final_price",
    "list_price": "initial_price",
    "product_rating": "rating",
    "product_brand": "brand",
    "product_url": "url",
    "category": "categories",
    "product_description": "description"
})


##### 6. Select matching columns

In [12]:
common_cols = [
    "discount",
    "categories",
    "url",
    "initial_price",
    "brand",
    "rating",
    "final_price",
    "currency",
    "upc",
    "description",
    "ingredients"
]

# Keep only existing ones
amazon = amazon[[c for c in common_cols if c in amazon.columns]]
walmart = walmart[[c for c in common_cols if c in walmart.columns]]


##### 7. Remove duplicate records

In [13]:
print("Amazon duplicate columns:", amazon.columns.duplicated().sum())
print("Walmart duplicate columns:", walmart.columns.duplicated().sum())

# Remove duplicate columns from Amazon
amazon = amazon.loc[:, ~amazon.columns.duplicated()]

# Double-check
print("Amazon duplicate columns after fix:", amazon.columns.duplicated().sum())


Amazon duplicate columns: 1
Walmart duplicate columns: 0
Amazon duplicate columns after fix: 0


##### 7. Add Platform Column

In [14]:
amazon["platform"] = "Amazon"
walmart["platform"] = "Walmart"
amazon.head(2)
walmart.head(2)

,discount,categories,url,initial_price,brand,rating,final_price,currency,upc,description,ingredients,platform
0,NaN,"[""Beauty"",""Makeup"",""Eye Makeup"",""Eye Shadow"",""...",https://www.walmart.com/ip/Laura-Mercier-Cavia...,NaN,Laura Mercier,4.0,22.9,USD,7.361500e+11,Laura Mercier Caviar Stick Eye Color Sugar Fro...,"Cyclopentasiloxane, trimethylsiloxysilicate, s...",Walmart
1,$22.92,"[""Home"",""Decor"",""Curtains & Window Treatments""...",https://www.walmart.com/ip/Exultantex-Grey-Bla...,70.8,Exultantex,4.6,47.9,USD,7.710780e+11,✨Soft triple weave fabric with a velvet-lik...,NaN,Walmart


##### 9. Combine Datasets

In [15]:
df = pd.concat([walmart, amazon], ignore_index=True)

# Check counts
print(df["platform"].value_counts())
print("Total rows:", df.shape[0])

platform
Amazon     1066
Walmart    1000
Name: count, dtype: int64
Total rows: 2066


In [16]:
df.isnull().sum()

discount         1112
categories        168
url                36
initial_price     740
brand             116
rating             80
final_price       512
currency           36
upc               968
description       303
ingredients      1916
platform            0
dtype: int64

##### 10. Handling Missing Values

In [17]:
# Convert to correct types
df['rating'] = pd.to_numeric(df['rating'], errors='coerce')
df['final_price'] = pd.to_numeric(df['final_price'], errors='coerce')
df['initial_price'] = pd.to_numeric(df['initial_price'], errors='coerce')
df['description'] = df['description'].astype(str)
df['categories'] = df['categories'].astype(str)

# Fill missing values safely
df['discount'] = (
    df['discount']
    .astype(str)                  # ensure it's string
    .str.replace('$', '', regex=False)  # remove $
)
df['discount'] = pd.to_numeric(df['discount'], errors='coerce')
df['discount'] = df['discount'].fillna(0)
df['rating'] = df['rating'].fillna(df['rating'].mean())
df['brand'] = df['brand'].fillna('Unknown')
df['final_price'] = df['final_price'].fillna(df['final_price'].median())
df['currency'] = df['currency'].fillna('USD')
df['initial_price'] = df['initial_price'].fillna(df['final_price'])
df['description'] = df['description'].fillna('No description available')
# Removed zero rating. rating is starting with 1 and ends with 5
df.loc[df['rating'] == 0, 'rating'] = 1.0

# Drop unnecessary columns safely
df = df.drop(columns=['ingredients', 'upc', 'url'], errors='ignore')

# Check missing values
df.isnull().sum()
df['discount'].head(4)

0     0.00
1    22.92
2     4.75
3    22.00
Name: discount, dtype: float64

##### 11. Handling Outliers for Walmart and Amazon

In [18]:
amazon_df = df[df['platform'] == 'Amazon']
walmart_df = df[df['platform'] == 'Walmart']

#  rating
q1 = amazon_df['rating'].quantile(0.25)
q3 = amazon_df['rating'].quantile(0.75)
iqr = q3 - q1
lower_bound = q1 - 1.5 * iqr
upper_bound = q3 + 1.5 * iqr

outliers = amazon_df[(amazon_df['rating'] < lower_bound) | (amazon_df['rating'] > upper_bound)]
print("Amazon rating outliers:\n", outliers)

q1 = walmart_df['rating'].quantile(0.25)
q3 = walmart_df['rating'].quantile(0.75)
iqr = q3 - q1
lower_bound = q1 - 1.5 * iqr
upper_bound = q3 + 1.5 * iqr
outliers = walmart_df[(walmart_df['rating'] < lower_bound) | (walmart_df['rating'] > upper_bound)]
print("Walmart rating outliers:\n", outliers)
# final_price
q1 = amazon_df['final_price'].quantile(0.25)
q3 = amazon_df['final_price'].quantile(0.75)
iqr = q3 - q1
lower_bound = q1 - 1.5 * iqr
upper_bound = q3 + 1.5 * iqr
outliers = amazon_df[(amazon_df['final_price'] < lower_bound) | (amazon_df['final_price'] > upper_bound)]
print("Amazon final_price outliers:\n", outliers)
q1 = walmart_df['final_price'].quantile(0.25)
q3 = walmart_df['final_price'].quantile(0.75)
iqr = q3 - q1
lower_bound = q1 - 1.5 * iqr
upper_bound = q3 + 1.5 * iqr
outliers = walmart_df[(walmart_df['final_price'] < lower_bound) | (walmart_df['final_price'] > upper_bound)]
print("Walmart final_price outliers:\n", outliers)
# initial_price
q1 = amazon_df['initial_price'].quantile(0.25)
q3 = amazon_df['initial_price'].quantile(0.75)
iqr = q3 - q1
lower_bound = q1 - 1.5 * iqr
upper_bound = q3 + 1.5 * iqr
outliers = amazon_df[(amazon_df['initial_price'] < lower_bound) | (amazon_df['initial_price'] > upper_bound)]
print("Amazon initial_price outliers:\n", outliers)
q1 = walmart_df['initial_price'].quantile(0.25)
q3 = walmart_df['initial_price'].quantile(0.75)
iqr = q3 - q1
lower_bound = q1 - 1.5 * iqr
upper_bound = q3 + 1.5 * iqr
outliers = walmart_df[(walmart_df['initial_price'] < lower_bound) | (walmart_df['initial_price'] > upper_bound)]
print("Walmart initial_price outliers:\n", outliers.shape)

#  Only outliers in walmart rating seem okay.
# No need to remove from df as they are valid data points.


Amazon rating outliers:
 Empty DataFrame
Columns: [discount, categories, initial_price, brand, rating, final_price, currency, description, platform]
Index: []
Walmart rating outliers:
      discount                                         categories  \
16       2.00  ["Clothing","Womens Clothing","Womens Activewe...   
29       0.59  ["Clothing","Womens Clothing","Womens Bras, Pa...   
55       3.99  ["Clothing","Womens Plus","Plus Size Activewea...   
71       0.80  ["Clothing","Womens Clothing","Womens Pants","...   
76      19.10  ["Premium Beauty","Premium Fragrance","Premium...   
..        ...                                                ...   
960      4.00  ["Clothing","Womens Clothing","Womens Swimsuit...   
962      2.30  ["Clothing","Womens Clothing","Womens Dresses"...   
981     11.80  ["Clothing","Mens Clothing","Mens Sweaters","M...   
990      0.90  ["Sports & Outdoors","Sports","Tennis & Racque...   
997      3.00  ["Clothing","Shoes","Womens Shoes","Womens San...   

In [19]:
(df['rating'] == 0).sum()

np.int64(0)

##### 11. Analysis

In [176]:
# Shape of dataset
df.shape
# Column information
df.info()
# First few rows
df.head()
df.describe()
df.groupby('platform').describe()
df.isnull().sum()
# Percentage of missing values
(df.isnull().sum() / len(df)) * 100



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2066 entries, 0 to 2065
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   discount       2066 non-null   float64
 1   categories     2066 non-null   object 
 2   initial_price  2066 non-null   float64
 3   brand          2066 non-null   object 
 4   rating         2066 non-null   float64
 5   final_price    2066 non-null   float64
 6   currency       2066 non-null   object 
 7   description    2066 non-null   object 
 8   platform       2066 non-null   object 
dtypes: float64(4), object(5)
memory usage: 145.4+ KB


discount         0.0
categories       0.0
initial_price    0.0
brand            0.0
rating           0.0
final_price      0.0
currency         0.0
description      0.0
platform         0.0
dtype: float64

##### 12. Universal Analysis

In [184]:
df['rating'].value_counts()

rating
1.000000    596
5.000000    223
4.600000    141
4.400000    131
4.500000    127
3.060338    112
4.700000    110
4.200000    100
4.300000     93
4.800000     85
4.000000     60
4.100000     46
3.000000     36
3.700000     34
3.800000     34
4.900000     32
3.900000     26
3.600000     18
3.300000     12
3.500000     10
2.000000      8
3.400000      6
2.800000      4
3.200000      4
2.500000      3
3.100000      3
2.900000      2
2.300000      2
2.700000      2
1.800000      1
1.300000      1
2.400000      1
1.500000      1
1.700000      1
2.600000      1
Name: count, dtype: int64

In [185]:
df['rating'].describe()

count    2066.000000
mean        3.334297
std         1.581401
min         1.000000
25%         1.000000
50%         4.200000
75%         4.600000
max         5.000000
Name: rating, dtype: float64

In [186]:
df['final_price'].describe()

count    2066.000000
mean       45.501332
std       148.598315
min         0.100000
25%        16.000000
50%        22.000000
75%        30.000000
max      3310.000000
Name: final_price, dtype: float64

In [187]:
df['final_price'].value_counts().head(4)

final_price
22.00    551
15.00     68
20.00     54
9.99      33
Name: count, dtype: int64

In [ ]:
df['discount'].describe()

count    2066.000000
mean        9.290503
std        45.641570
min         0.000000
25%         0.000000
50%         0.000000
75%         6.195000
max       958.000000
Name: discount, dtype: float64

##### 13.Bivariate Analysis

In [ ]:
# Platform vs price analysis
df.groupby('platform')['final_price'].mean()


platform
Amazon     52.535358
Walmart    38.003060
Name: final_price, dtype: float64

In [172]:
# Plateform vs rating analysis
df.groupby('platform')['rating'].mean()

platform
Amazon     2.391987
Walmart    4.338800
Name: rating, dtype: float64

In [20]:
# Rating vs discount analysis
df.groupby('rating')['discount'].mean()


rating
1.000000     0.213456
1.300000     0.590000
1.500000     0.000000
1.700000     0.000000
1.800000     3.990000
2.000000     1.195000
2.300000     1.500000
2.400000     0.000000
2.500000     6.000000
2.600000     0.000000
2.700000     2.000000
2.800000     1.200000
2.900000    17.000000
3.000000     5.175833
3.060338     0.000000
3.100000     6.566667
3.200000     3.820000
3.300000     5.427500
3.400000     3.181667
3.500000     5.360000
3.600000     3.967222
3.700000    10.641471
3.800000     2.675000
3.900000     4.626923
4.000000    13.320000
4.100000     7.046087
4.200000     8.391800
4.300000    14.737742
4.400000    15.619924
4.500000    13.695118
4.600000    33.470496
4.700000    19.749545
4.800000    13.404118
4.900000    38.564375
5.000000     7.182152
Name: discount, dtype: float64

In [21]:
# Brand vs Price (Top Brands)
df.groupby('brand')['final_price'].mean().sort_values(ascending=False).head(10)


brand
DIAMOND DOMAIN        3310.000
Zayan Rugs            2240.000
三科 光平                 2001.000
EternalDia            1696.000
Westintrends          1690.000
Clara Pucci           1581.865
KarParts360           1110.880
Men's Fine Jewelry    1049.990
Lovechild Masaba       950.000
Chief                  791.700
Name: final_price, dtype: float64

##### 14. Multivariate Analysis

In [ ]:
# Pivot Table for Platform and currency
pd.pivot_table(
    df,
    values='final_price',
    index='platform',
    columns='currency',
    aggfunc='mean'
).head()


currency,each Rick has his own Morty,CAD,EUR,GPB,INR,JPY,USD,"color"""":""""64# Teal + Blackout Lining""""","color"""":""""Blue Marble""""","color"""":""""Dark Heather Grey""""",...,"{""""asin"""":""""B085DNDK2V""""","{""""asin"""":""""B08GPRJ65F""""","{""""asin"""":""""B0CQGH4FDL""""","{""""asin"""":""""B0D6FMHYX4""""","{""""asin"""":""""B0D6VNMS1Y""""","{""""asin"""":""""B0DP9P4GNH""""","{""""asin"""":""""B0DVVFPGZM""""","{""""asin"""":""""B0FK1TC9KJ""""","{""""asin"""":""""B0G8YK3LMY""""","{""""asin"""":""""MC_Assembly_1#B07537PB9W"""""
platform,,,,,,,,,,,,,,,,,,,,,
Amazon,22.0,22.275,30.572,25.826,709.0,2001.0,50.619451,22.0,22.0,22.0,...,22.0,22.0,22.0,22.0,22.0,22.0,22.0,22.0,22.0,22.0
Walmart,NaN,NaN,NaN,NaN,NaN,NaN,38.003060,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Plvot table for platform, categories, and final price
pd.pivot_table(
    df,
    values='final_price',
    index='platform',
    columns='categories',
    aggfunc='mean'
).head()


categories,"Green""""",daughter of Jerry and Beth,"[""Appliances"",""Parts & Accessories"",""Refrigerator Parts & Accessories"",""Handles""]","[""Appliances"",""Parts & Accessories"",""Refrigerator Parts & Accessories"",""Motors""]","[""Appliances"",""Parts & Accessories""]","[""Arts Crafts & Sewing"",""Fabric"",""Shop All Fabric""]","[""Arts Crafts & Sewing"",""Scrapbooking"",""Stickers""]","[""Arts, Crafts & Sewing"",""Beading & Jewelry Making"",""Beads & Bead Assortments""]","[""Arts, Crafts & Sewing"",""Fabric""]","[""Arts, Crafts & Sewing"",""Model & Hobby Building"",""Model Kits"",""Figure Kits""]",...,"name"""":""""64# Teal + Blackout Lining 56\""""W x 108\""""L (Pack of 1)""""","name"""":""""December Birthstone 10.25""""","name"""":""""Men's Big and Tall Dark Heather Grey 6X Tall""""","name"""":""""S (18-21 inch suitcase) Blue Marble""""",nan,"size"""":""""34\""""W x 36\""""L (Pack of 2)""""","size"""":""""Medium""""","size"""":""""Super King- 3 PC""""","unit_price"""":null}","{""""asin"""":""""B0FH1BWDNN"""""
platform,,,,,,,,,,,,,,,,,,,,,
Amazon,22.0,22.0,40.99,15.99,8.99,NaN,NaN,15.995,20.992,10.98,...,22.0,22.0,22.0,22.0,33.335,22.0,22.0,22.0,22.0,22.0
Walmart,NaN,NaN,NaN,NaN,NaN,10.8,3.21,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


##### 15. Correlation Analysis

In [22]:
num_cols = ['final_price','initial_price','rating','discount']

corr_matrix = df[num_cols].corr()
corr_matrix


,final_price,initial_price,rating,discount
final_price,1.000000,0.969404,-0.031069,0.388557
initial_price,0.969404,1.000000,0.011508,0.597535
rating,-0.031069,0.011508,1.000000,0.148237
discount,0.388557,0.597535,0.148237,1.000000


#### 16. Exploratory Data Analysis (EDA) – Key Insights

##### 1. Univariate Analysis (Single Variable Analysis)
- Rating Distribution
    * Total observations: 2066
    * Mean rating: 3.33
    * Median rating: 4.20
    * Minimum rating: 1.0
    * Maximum rating: 5.0

    - Insights:

        - Ratings are concentrated toward the higher range (4.0–5.0).

        - The median (4.2) is higher than the mean (3.33), indicating a left-skewed distribution.

        - A large number of products received a rating of 1, mainly due to replacement of missing values with 1.

        - This impacts the overall average rating and slightly lowers the mean.

- Final Price Distribution
    * Mean price: $45.50
    * Median price: $22.00
    * Maximum price: $3310
    * Standard deviation: 148.59

    - Insights:

        - Most products are priced below $30, as shown by the median.

        - A small number of very expensive products increase the mean price.

        - High standard deviation indicates wide price variation.

        - The dataset contains several high-priced outliers.

- Discount Distribution
    * Mean discount: $9.29
    * Median discount: $0.00
    * Maximum discount: $958
    * 50% of products have no discount

    - Insights:

        - More than half of the products do not offer any discount.

        - Discounts are highly skewed, with a few products having very large discounts.

        - Most discounts are concentrated in the lower range (0–6).

- Most Common Prices
    * Top occurring prices:
        * $22.00 → 551 products
        * $15.00 → 68 products
        * $20.00 → 54 products
        * $9.99 → 33 products

    - Insights:

        - Pricing is clustered around popular psychological price points.

        - $22 and $20 appear to be dominant pricing strategies.

##### 2. Bivariate Analysis (Relationship Between Two Variables)

- Platform vs Final Price

| Platform | Average Price    |
|----------|------------------|
| Amazon   | $52.54           |
| Walmart  | $38.00           |

- Insights:

    - Amazon products are generally more expensive than Walmart products.

    - Walmart focuses more on budget-friendly items.

    - Amazon appears to have a wider premium product range.

- Platform vs Rating
| Platform | Average Rating   |
|----------|------------------|
| Amazon   | 2.39             |
| Walmart  | 4.34             |

- Insights:

    - Walmart products receive significantly higher ratings.

    - Amazon’s lower average rating is influenced by many replaced missing values.

    - Customer satisfaction appears higher on Walmart.

- Rating vs Discount

    - Products with higher ratings (4.5–5.0) tend to have moderate to high discounts.

    - Some low-rated products also receive discounts.

- Insights:

    - Discounts are not strongly dependent on ratings.

    - Well-rated products are still discounted, likely for promotional purposes.

- Brand vs Final Price

    - Top expensive brands:

        - DIAMOND DOMAIN → $3310

        - Zayan Rugs → $2240

        - EternalDia → $1696

        - Westintrends → $1690

Insights:

- Certain premium brands dominate the high-price segment.

- Brand reputation strongly influences pricing.

##### 3. Multivariate Analysis (Multiple Variables)

- Platform and Currency

    - Amazon products appear in multiple currencies (USD, INR, EUR, JPY, etc.).

    - Walmart products are mainly in USD.

- Insights:

    - Amazon has wider international presence.

    - Walmart’s dataset is more region-focused.

- Platform, Category, and Price

    - Amazon covers a wide range of categories such as electronics, fashion, home, and accessories.

    - Walmart has fewer categories represented.

- Insights:

    - Amazon has greater product diversity.

    - Walmart focuses on selected categories.

##### 4. Correlation Analysis
|   Variables                         |   Correlation                           |
|-------------------------------------|-----------------------------------------|
|   Final – Initial	Price             |   0.97                                  |
|   Initial – Discount	              |   0.60                                  |
|   Final – Discount	              |   0.39                                  |
|   Rating – Price	                  |   -0.03                                 |
|   Rating – Discount                 |   0.15	                                |

- Key Observations
    - Final Price vs Initial Price (0.97)

        * Very strong positive correlation.

        * Final prices closely follow original prices.

        * Discounts are usually proportional.

    - Initial Price vs Discount (0.60)

        * Higher-priced products receive higher discounts.

        * Expensive items are often promoted more.

    - Rating vs Price (-0.03)

        * Almost no relationship.

        * Expensive products are not necessarily better rated.

    - Rating vs Discount (0.15)

        - Weak positive relationship.

        - Discounts have minimal impact on ratings.

##### 5. Overall Findings and Business Insights

Key Findings

- Most products are low to moderately priced, with a few extreme outliers.

- Amazon products are generally more expensive than Walmart.

- Walmart products have higher average ratings.

- More than half of the products have no discount.

- Price is strongly influenced by brand and original price.

- Ratings are weakly related to price and discount.

Limitations

- Missing ratings were replaced with 1, which may bias results.

- Discount values show extreme outliers.

- Some categorical fields contain noisy or unstructured data.


16. Save Cleaned DataFrame to CSV

In [23]:
# Save cleaned dataset
df.to_csv("cleaned_amazon_walmart_data.csv", index=False)

print("CSV file saved successfully!")


CSV file saved successfully!
